# Práctica de Visión por Computadora para Robótica con scikit-image

## Objetivo

Desarrollar habilidades en procesamiento de imágenes aplicadas a robótica usando scikit-image, simulando tareas típicas que un robot necesita realizar.

## Configuración Inicial


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, measure, morphology, segmentation
from skimage.color import rgb2gray, rgb2hsv
from skimage.feature import canny, corner_harris, corner_peaks
from skimage.transform import hough_line, hough_line_peaks, hough_circle
from skimage.util import random_noise
import cv2  # Para algunas funciones complementarias

## Ejercicio 1: Detección de Objetos por Color (Clasificación de Piezas)

**Escenario**: Un robot debe clasificar piezas de diferentes colores en una banda transportadora.

**Tareas**:

1. Carga una imagen con objetos de diferentes colores
2. Convierte a espacio de color HSV
3. Crea máscaras para detectar objetos rojos, azules y verdes
4. Cuenta el número de objetos de cada color
5. Calcula el centroide de cada objeto


In [3]:
def detectar_objetos_por_color(imagen):
    """
    Detecta y clasifica objetos por color
    """
    # Tu código aquí
    pass

# Imagen de prueba (puedes usar una imagen sintética o real)
# imagen = io.imread('piezas_colores.jpg')

In [9]:
def detectar_objetos_por_color(imagen_path):
    """
    Detecta y clasifica objetos por color, calcula centroides.
    Retorna un diccionario con los resultados.
    """
    try:
        imagen = io.imread(imagen_path)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta '{imagen_path}'. Asegúrate de que la ruta sea correcta.")
        return None
    
    # Convertir a HSV
    hsv = rgb2hsv(imagen)
    h, s, v = hsv[:, :, 0], hsv[:, :, 1], hsv[:, :, 2]

    resultados = {
        'rojo': {'count': 0, 'centroids': []},
        'verde': {'count': 0, 'centroids': []},
        'azul': {'count': 0, 'centroids': []}
    }

    # Rangos de color en HSV
    # Nota: El matiz (Hue) en scikit-image está en [0, 1]
    # Rangos para el rojo (es circular, así que necesita dos rangos)
    rojo_mask1 = (h > 0.95) | (h < 0.05)
    rojo_mask2 = (s > 0.4)
    rojo_mask = rojo_mask1 & rojo_mask2

    # Rangos para el verde
    verde_mask = (h > 0.25) & (h < 0.45) & (s > 0.4)

    # Rangos para el azul
    azul_mask = (h > 0.55) & (h < 0.7) & (s > 0.4)

    # Procesa cada color
    for color, mask in [('rojo', rojo_mask), ('verde', verde_mask), ('azul', azul_mask)]:
        labeled_mask = measure.label(mask)
        props = measure.regionprops(labeled_mask)
        
        # Filtrar objetos pequeños y contar
        for prop in props:
            if prop.area > 50: # Evita el ruido
                resultados[color]['count'] += 1
                resultados[color]['centroids'].append(prop.centroid)

    # Visualización (opcional)
    fig, ax = plt.subplots(1, 4, figsize=(20, 5))
    ax[0].imshow(imagen)
    ax[0].set_title('Original')
    ax[0].axis('off')

    ax[1].imshow(rojo_mask, cmap='gray')
    ax[1].set_title('Máscara Roja')
    ax[1].axis('off')
    
    ax[2].imshow(verde_mask, cmap='gray')
    ax[2].set_title('Máscara Verde')
    ax[2].axis('off')

    ax[3].imshow(azul_mask, cmap='gray')
    ax[3].set_title('Máscara Azul')
    ax[3].axis('off')
    plt.show()

    print("Resultados de la detección de objetos:")
    for color, data in resultados.items():
        print(f"  - {color.capitalize()}: {data['count']} objetos, Centroides: {data['centroids']}")
    
    return resultados

# Ejemplo de uso:
# Crea una imagen sintética para la prueba
imagen_sintetica = np.zeros((200, 300, 3), dtype=np.uint8)
# Dibuja objetos de colores
cv2.circle(imagen_sintetica, (50, 50), 20, (255, 0, 0), -1) # Rojo
cv2.circle(imagen_sintetica, (150, 50), 25, (0, 255, 0), -1) # Verde
cv2.rectangle(imagen_sintetica, (200, 30), (250, 80), (0, 0, 255), -1) # Azul
cv2.circle(imagen_sintetica, (100, 150), 30, (255, 0, 0), -1) # Rojo

# Guarda la imagen sintética
io.imsave('piezas_colores.png', imagen_sintetica)

# Llama a la función con la imagen de prueba
# detectar_objetos_por_color('piezas_colores.png')

**Criterios de evaluación**:

- Precisión en la detección (>90%)
- Robustez ante variaciones de iluminación
- Cálculo correcto de centroides


## Ejercicio 2: Navegación por Líneas (Seguimiento de Trayectoria)

**Escenario**: Un robot móvil debe seguir una línea en el suelo para navegar.

**Tareas**:

1. Detecta líneas usando la transformada de Hough
2. Calcula el ángulo de la línea principal
3. Determina si el robot debe girar izquierda/derecha
4. Estima la distancia al centro de la línea


In [4]:
def seguir_linea(imagen):
    """
    Procesa imagen para seguimiento de línea
    Retorna: ángulo_giro, distancia_centro
    """
    # Preprocesamiento
    gray = rgb2gray(imagen)

    # Detección de bordes
    edges = canny(gray, sigma=2, low_threshold=0.1, high_threshold=0.2)

    # Tu código para Hough Transform aquí

    return angulo_giro, distancia_centro

In [3]:
def seguir_linea(imagen_path):
    """
    Procesa la imagen para seguimiento de línea.
    Retorna: angulo_giro, distancia_centro.
    """
    try:
        imagen = io.imread(imagen_path)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta '{imagen_path}'.")
        return None, None

    # Preprocesamiento
    gray = rgb2gray(imagen)
    edges = canny(gray, sigma=2, low_threshold=0.1, high_threshold=0.2)

    # Transformada de Hough
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
    h, theta, d = hough_line(edges, theta=tested_angles)

    # Encontrar picos de la transformada de Hough
    picos_d, picos_theta = hough_line_peaks(h, theta, d, num_peaks=1)
    
    angulo_giro = 0
    distancia_centro = 0
    
    if len(picos_theta) > 0:
        # El ángulo de la línea principal
        line_angle_rad = picos_theta[0]
        
        # Calcular el ángulo de giro en grados
        # 0 grados es recto, positivo a la derecha, negativo a la izquierda
        # np.pi / 2 es 90 grados
        angulo_giro = np.degrees(np.pi/2 - line_angle_rad)
        
        # Calcular la distancia al centro de la imagen
        rho = picos_d[0]
        img_center_x = imagen.shape[1] / 2
        # La distancia al centro se puede estimar como el desplazamiento del centro
        # de la línea detectada (rho) con respecto al centro de la imagen.
        distancia_centro = rho - (img_center_x * np.cos(line_angle_rad))
        
        # Visualización (opcional)
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].imshow(imagen, cmap='gray')
        ax[0].set_title('Original')
        ax[0].axis('off')
        
        ax[1].imshow(edges, cmap='gray')
        ax[1].set_title('Bordes')
        ax[1].set_ylim((edges.shape[0], 0))
        ax[1].set_axis_off()
        
        # Dibuja la línea detectada
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d, num_peaks=5)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - edges.shape[1] * np.cos(angle)) / np.sin(angle)
            ax[1].plot((0, edges.shape[1]), (y0, y1), '-r')
        
        plt.tight_layout()
        plt.show()

    print(f"Ángulo de giro (grados): {angulo_giro:.2f}")
    print(f"Distancia al centro de la línea (píxeles): {distancia_centro:.2f}")
    
    return angulo_giro, distancia_centro

# Ejemplo de uso:
# Crea una imagen sintética de una línea
line_image = np.zeros((200, 200), dtype=np.uint8)
cv2.line(line_image, (20, 20), (180, 180), 255, 5)
io.imsave('linea_prueba.png', line_image)

# seguir_linea('linea_prueba.png')

**Reto adicional**: Maneja intersecciones y bifurcaciones.

## Ejercicio 3: Detección y Medición de Objetos (Control de Calidad)

**Escenario**: Un robot industrial debe medir dimensiones de piezas manufacturadas.

**Tareas**:

1. Segmenta objetos del fondo
2. Calcula área, perímetro y dimensiones principales
3. Detecta defectos (agujeros, irregularidades)
4. Clasifica piezas como "aprobadas" o "defectuosas"

In [5]:
def control_calidad(imagen, tolerancia_area=0.1):
    """
    Analiza piezas para control de calidad
    """
    # Segmentación
    gray = rgb2gray(imagen)
    thresh = filters.threshold_otsu(gray)
    binary = gray > thresh

    # Análisis morfológico
    cleaned = morphology.remove_small_objects(binary, min_size=100)

    # Tu código para mediciones aquí

    return resultados_medicion

In [4]:
def control_calidad(imagen_path, tolerancia_area=0.1):
    """
    Analiza piezas para control de calidad.
    """
    try:
        imagen = io.imread(imagen_path)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta '{imagen_path}'.")
        return None
    
    # Segmentación
    gray = rgb2gray(imagen)
    thresh = filters.threshold_otsu(gray)
    binary = gray > thresh
    
    # Análisis morfológico (limpiar ruido)
    cleaned = morphology.remove_small_objects(binary, min_size=100)
    labeled = measure.label(cleaned)
    
    resultados_medicion = []
    
    # Análisis de propiedades de las regiones
    props = measure.regionprops(labeled)
    
    for i, prop in enumerate(props):
        area = prop.area
        perimetro = prop.perimeter
        largo = prop.major_axis_length
        ancho = prop.minor_axis_length
        
        # Evita la división por cero
        circularidad = (4 * np.pi * area) / (perimetro ** 2) if perimetro > 0 else 0
        aspect_ratio = largo / ancho if ancho > 0 else 0
        
        # Detección de agujeros
        holes = np.sum(labeled == (i+1)) - np.sum(prop.image) # Área de la región - Área del 'prop'
        # Una forma más sencilla para el número de agujeros
        holes = prop.num_pixels - prop.filled_area # No es 100% preciso, pero funciona
        
        
        # Clasificación simple
        clasificacion = "Aprobada"
        if circularidad < 0.8: # Umbral de circularidad bajo
            clasificacion = "Defectuosa (Irregular)"
        if holes > 0:
            clasificacion = "Defectuosa (Agujeros)"
        
        resultados_medicion.append({
            'ID': i + 1,
            'Area (px^2)': area,
            'Perimetro (px)': perimetro,
            'Aspect Ratio': aspect_ratio,
            'Circularidad': circularidad,
            'Agujeros': holes,
            'Clasificacion': clasificacion
        })

    # Visualización de los objetos etiquetados (opcional)
    plt.imshow(labeled, cmap=plt.cm.nipy_spectral)
    plt.title("Objetos segmentados")
    plt.show()

    print("Resultados de Control de Calidad:")
    for res in resultados_medicion:
        print(f"--- Pieza ID {res['ID']} ({res['Clasificacion']}) ---")
        print(f"  - Área: {res['Area (px^2)']:.2f} px²")
        print(f"  - Perímetro: {res['Perimetro (px)']:.2f} px")
        print(f"  - Relación Aspecto: {res['Aspect Ratio']:.2f}")
        print(f"  - Circularidad: {res['Circularidad']:.2f}")
        print(f"  - Agujeros: {res['Agujeros']}")

    return resultados_medicion

# Ejemplo de uso:
# Crea una imagen sintética para control de calidad
qc_image = np.zeros((200, 200), dtype=np.uint8)
cv2.circle(qc_image, (60, 60), 40, 255, -1) # Círculo aprobado
cv2.rectangle(qc_image, (120, 120), (180, 180), 255, -1) # Cuadrado
cv2.circle(qc_image, (150, 50), 20, 255, -1) # Círculo pequeño para ruido
cv2.circle(qc_image, (60, 60), 10, 0, -1) # Agujero
io.imsave('piezas_calidad.png', qc_image)

# control_calidad('piezas_calidad.png')

**Métricas a calcular**:

- Área en píxeles y mm²
- Relación aspecto (largo/ancho)
- Circularidad: 4π×área/perímetro²
- Número de agujeros


## Ejercicio 4: Reconocimiento de Formas Geométricas

**Escenario**: Un robot debe clasificar y manipular objetos según su forma.

**Tareas**:

1. Detecta contornos de objetos
2. Clasifica formas: círculo, cuadrado, triángulo, rectángulo
3. Calcula orientación para agarre robótico
4. Determina puntos de agarre óptimos


In [6]:
def clasificar_formas(imagen):
    """
    Clasifica objetos por su forma geométrica
    """
    # Detección de contornos
    gray = rgb2gray(imagen)
    edges = canny(gray, sigma=1)

    # Análisis de forma
    contours = measure.find_contours(edges, 0.5)

    formas_detectadas = []

    for contour in contours:
        # Tu código para clasificación aquí
        # Usa aproximación poligonal, análisis de curvatura, etc.
        pass

    return formas_detectadas

In [5]:
def clasificar_formas(imagen_path):
    """
    Clasifica objetos por su forma geométrica.
    """
    try:
        imagen = io.imread(imagen_path)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta '{imagen_path}'.")
        return None
        
    # Preprocesamiento y detección de contornos
    gray = rgb2gray(imagen)
    edges = canny(gray, sigma=1.5)
    
    # Encontrar contornos
    contours = measure.find_contours(edges, 0.5)

    formas_detectadas = []

    # Se usa OpenCV para la aproximación de contornos, ya que es muy eficaz.
    # Convierte el contorno de skimage a un formato compatible con cv2
    for contour in contours:
        contour_cv2 = np.array(contour, dtype=np.float32)
        
        # Aproximación de Douglas-Peucker
        epsilon = 0.04 * cv2.arcLength(contour_cv2, True)
        approx = cv2.approxPolyDP(contour_cv2, epsilon, True)
        num_vertices = len(approx)
        
        # Calcular métricas para clasificar
        area = cv2.contourArea(approx)
        perimetro = cv2.arcLength(approx, True)
        
        # Evitar el ruido y contornos vacíos
        if perimetro > 10 and area > 100:
            shape = "Desconocida"
            
            # Clasificación simple por número de vértices
            if num_vertices == 3:
                shape = "Triángulo"
            elif num_vertices == 4:
                # Distinguir entre cuadrado y rectángulo por relación de aspecto
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = float(w)/h
                if 0.95 < aspect_ratio < 1.05:
                    shape = "Cuadrado"
                else:
                    shape = "Rectángulo"
            elif num_vertices > 7:
                # Usar circularidad para círculos
                if perimetro > 0:
                    circularidad = (4 * np.pi * area) / (perimetro ** 2)
                    if circularidad > 0.8:
                        shape = "Círculo"
            
            # Orientación (usando Bounding Box rotado)
            rect = cv2.minAreaRect(contour_cv2)
            angulo_rotacion = rect[-1]
            
            # Puntos de agarre (simples: centroides)
            M = cv2.moments(contour_cv2)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                centroid = (cx, cy)
            else:
                centroid = (0,0)

            formas_detectadas.append({
                'forma': shape,
                'area': area,
                'vertices': num_vertices,
                'orientacion (deg)': angulo_rotacion,
                'centroide': centroid
            })

    print("Resultados de la clasificación de formas:")
    for i, forma in enumerate(formas_detectadas):
        print(f"--- Objeto {i+1} ---")
        print(f"  - Forma: {forma['forma']}")
        print(f"  - Área: {forma['area']:.2f}")
        print(f"  - Vértices: {forma['vertices']}")
        print(f"  - Orientación: {forma['orientacion (deg)']:.2f} grados")
        print(f"  - Centroide: {forma['centroide']}")
        
    return formas_detectadas

# Ejemplo de uso:
# Crea una imagen sintética
forms_image = np.zeros((200, 200), dtype=np.uint8)
cv2.circle(forms_image, (50, 50), 30, 255, -1) # Círculo
cv2.rectangle(forms_image, (120, 120), (180, 180), 255, -1) # Cuadrado
io.imsave('formas_prueba.png', forms_image)

# clasificar_formas('formas_prueba.png')

**Algoritmos sugeridos**:

- Aproximación de Douglas-Peucker
- Análisis de momentos de Hu
- Descriptores de Fourier